# Configuration

The `logging` package contains many facilities to set up the configuration for it. There is even a separate module for it, `logging.config`. This page is focused on such options.

Check [documentation of the `logging.config` module](https://docs.python.org/3/library/logging.config.html#module-logging.config).

In [4]:
import logging

## `ini` file

You can store your configuration in the `.ini` files.

Read more about `.ini` files:

- [configparser](https://docs.python.org/3/library/configparser.html#module-configparser) python package;
- [yaml vs ini](https://www.honeybadger.io/blog/python-ini-vs-yaml/) page that describes structure of the ini and yaml files.

Here is example of suck `.ini` file.

In [5]:
%%writefile configuration_files/example_logging.ini
[loggers]
keys=root,simpleExample

[handlers]
keys=consoleHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler

[logger_simpleExample]
level=DEBUG
handlers=consoleHandler
qualname=simpleExample
propagate=0

[handler_consoleHandler]
class=StreamHandler
level=DEBUG
formatter=simpleFormatter
args=(sys.stdout,)

[formatter_simpleFormatter]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s

Overwriting configuration_files/example_logging.ini


You can use the `logging.config.fileConfig` function to load the configuration file - all it's entities will be added to your program. The following cell uses the config from the previous cell to define it's loggers.

In [7]:
logging.config.fileConfig('configuration_files/example_logging.ini')
logger = logging.getLogger('simpleExample')
logger.debug("test")

2024-05-07 15:34:09,986 - simpleExample - DEBUG - test
